# Introduction to Gurobi Python API

###### Gurobi Days Digital
###### June 24, 2026
###### Maliheh Aramon, PhD, Optimization Engineer
###### aramon@gurobi.com

## How to Run the Notebook locally?
- Visit [Gurobi modeling-examples](https://github.com/Gurobi/modeling-examples) repository 
- Clone the repository which contains this notebook and other examples or download it
by clicking [here](https://github.com/Gurobi/modeling-examples/archive/refs/heads/master.zip)
- Navigate to folder __intro_to_gurobipy__
- [Start Jupyter Notebook Server](https://docs.jupyter.org/en/latest/running.html#id2)
- Open the notebook in Jupyter Notebook
- The notebook will install the gurobipy package and other dependencies. The Gurobi pip package includes a size-limited trial license that will allow you to run the notebook

In [ ]:
%pip install "gurobipy>=10.0"

# Install other dependencies
%pip install numpy
%pip install scipy
%pip install pandas

# Gurobi Python API

Gurobi Python API, also known as _gurobipy_ is the most popular Gurobi API because it allows building the model with
- individual variables and constraints like other Guorbi's object-oriented APIs such as C, C++, Java, and .NET
- matrices like other Gurobi's matrix-oriented interfaces such as MATLAB and R
- more detailed mathematical syntax like traditional modeling languages

In this session, we will walk you through the basics of the Gurobi Python API.

## How to Install gurobipy?

There are two main approaches to install gurobipy on any operating systems such as Linux, Windows, or macOS.

- Pip
- Conda

Relevant Knowledge Base (KB) articles:
- [Which Python versions are supported by Gurobi?](https://support.gurobi.com/hc/en-us/articles/360013195212)
- [How do I install Gurobi for Python?](https://support.gurobi.com/hc/en-us/articles/360044290292-How-do-I-install-Gurobi-for-Python-)

## Optimization Models 

The canonical form of optimization models Gurobi can handle is given below:

\begin{align}
\mbox{Model P:} ~~~~ \mbox{minimize} \quad & x^T Q x + c^T x + d &  \notag \\
\mbox{subject to} \quad & Ax = b & \notag & \notag \\
                        & x^T Q_i x + c_i^T x \leq d_i & \forall i \in I \notag \\
                        & l \leq x \leq u & \notag \\
                        & x_j \in \mathbb{Z} & \forall j \in J 
\end{align}

In this session, we would learn how to map the above math constructs to code using gurobipy.

Each mathematical model has four main elements: __Data + Decision variables + Constraints + Objective function(s)__

__Data__:
\begin{align}
& \mbox{Sets:}~~ I, J & \notag \\ 
& \mbox{Coefficients:}~~ Q, c, A, Q_i, c_i \notag &\\ 
& \mbox{RHS values:}~~ b, d_i & \notag \\ 
& \mbox{Lower and upper bounds:}~~ l, u & \notag \\ 
& \mbox{Constants:}~~ d & \notag \\
& \mbox{Operators:} & \notag \\
& ~~~~~~ \mbox{Arithmetic}~ (+, -, *, \div)& \notag \\
& ~~~~~~ \mbox{Constraint operators} ~(\geq, \leq, =) & \notag
\end{align}

## Simple Example
Let us start with a simple example:

\begin{align}
\mbox{maximize} \quad & x + y + 2z \notag \\
\mbox{subject to} \quad & x + 2y + 3z \leq 4 \notag \\
                        & x + y \geq 1 \notag \\
                        & x, y, z \in \{0, 1\} \notag
\end{align}

In [ ]:
# Import gurobipy package as gp for convenience
import gurobipy as gp

# GRB is the list of all Gurobi constants
from gurobipy import GRB

# Create a Gurobi environment and a model object
with gp.Env() as env, gp.Model("simple-example", env=env) as model:
    # Define decision variables
    x = model.addVar(vtype=GRB.BINARY, name="x")
    y = model.addVar(vtype=GRB.BINARY, name="y")
    z = model.addVar(vtype=GRB.BINARY, name="z")

    # Define constraints
    model.addConstr(x + 2 * y + 3 * z <= 4, name="c0")
    model.addConstr(x + y >= 1, name="c1")

    # Define objective
    model.setObjective(x + y + 2 * z, sense=GRB.MAXIMIZE)

    # Optimize model
    model.optimize()

    print("******* Solution *******")
    for var in model.getVars():
        print(f"{var.VarName}: {var.X}")
    print("************************")

## Python Data Structures

- Tuple: An ordered, compound grouping that cannot be modified once it is created and it is ideal for representing multi dimensional subscripts.
    ```
        ("city_0", "city_1")
    ```
- List: An ordered group, so each item is indexed. Lists can be modified by adding, deleting or sorting elements.
    ```
        ["city_0", "city_1", "city_2"]
     ```
- Set: An unordered group of unique elements. Sets can only be modified by adding or deleting.
    ```
        {"city_0", "city_1", "city_2"}
    ```
- Dictionary: A key-value pair mapping that is ideal for representing indexed data such as cost, demand, capacity.
    ```
        damand = {"city_0": 100, "city_1": 50, "city_2": 40}
    ```

 ## Extented Data Structures in Gurobi Python API
 
 - [tuplelist()](https://www.gurobi.com/documentation/current/refman/py_tuplelist.html)
     - A sub-class of Python list 
     - Important methods to build sub-lists efficiently 
         - [tuplelist.select(pattern)](https://www.gurobi.com/documentation/current/refman/py_tuplelist_select.html) --> tuplelist()
     
 
 
- [tupledict()](https://www.gurobi.com/documentation/current/refman/py_tupledict.html)
    - A sub-class of Python dict
    - The keys of a tupledict() are stored as tuplelist() and the values are Gurobi variable objects
    - Important methods to build linear expressions efficiently:
        - [tupledict.select(pattern)](https://www.gurobi.com/documentation/current/refman/py_tupledict_select.html) --> List
        - [tupledict.sum(pattern)](https://www.gurobi.com/documentation/current/refman/py_tupledict_sum.html) --> LinExpr()
        - [tupledict.prod(coeff, pattern)](https://www.gurobi.com/documentation/current/refman/py_tupledict_prod.html) --> LinExpr()
    
    
- [multidict()](https://www.gurobi.com/documentation/current/refman/py_multidict.html): A convenience function to define multiple dictionaries in one statement.

In [ ]:
import gurobipy as gp
from gurobipy import GRB

data = gp.tupledict(
    [
        (("a", "b", "c"), 3),
        (("a", "c", "b"), 4),
        (("b", "a", "c"), 5),
        (("b", "c", "a"), 6),
        (("c", "a", "b"), 7),
        (("c", "b", "a"), 3),
    ]
)
print(f"data: {data}")

In [ ]:
print("\nTuplelist:")
keys = gp.tuplelist(data.keys())
print(f"\tselect: {keys.select('a', '*', '*')}")

In [ ]:
print("\nTupledict:")
print(f"\tselect  : {data.select('a', '*', '*')}")
print(f"\tsum     : {data.sum('*', '*', '*')}")
coeff = {("a", "c", "b"): 6, ("b", "c", "a"): -4}
print(f"\tprod    : {data.prod(coeff, '*', 'c', '*')}")

In [ ]:
arcs, capacity, cost = gp.multidict(
    {
        ("Detroit", "Boston"): [100, 7],
        ("Detroit", "New York"): [80, 5],
        ("Detroit", "Seattle"): [120, 4],
        ("Denver", "Boston"): [120, 8],
        ("Denver", "New York"): [120, 11],
        ("Denver", "Seattle"): [120, 4],
    }
)
print("\nMultidict:")
print(f"\tcapacity: {capacity}")
print("\n")
print(f"\tcost: {cost}")

## [Environments](https://www.gurobi.com/documentation/current/refman/py_env2.html)

```
+--------------------------------------------+
| Environment                                |
| +----------------------------------------+ |
| | Model                                  | |
| | +------+ +-----------+ +-------------+ | | 
| | | Data | | Variables | | Constraints | | |
| | +------+ +-----------+ +-------------+ | |
| | +-----------+                          | |
| | | Objective |                          | |
| | +-----------+                          | |
| +----------------------------------------+ |
+--------------------------------------------+  
```

Python API has a default environment which is used by default unless a new environment is created and explicitly passed to the routines that require an environment.

The main reason to create an environment is to have control over when your application starts using Gurobi and when it stops using it! 
- Using remote resources such as floating, cloud, or compute server licenses for optimization
- Garbage collection when using Jupyter notebooks

Note: It is better to create new environments via the [context manager](https://support.gurobi.com/hc/en-us/articles/4424054948881-How-do-I-manage-Gurobi-environments-in-gurobipy).

## [Model](https://www.gurobi.com/documentation/current/refman/py_model.html)
Model building in the [Python API](https://www.gurobi.com/documentation/current/refman/py_python_api_overview.html) is object oriented. The reference manual contains [a full list of methods](https://www.gurobi.com/documentation/current/refman/py_python_api_details.html#sec:Python-details) on a model object. 

The signature for constructing a model object is:
```
Model(name="", env=defaultEnv)
```

In [ ]:
import gurobipy as gp

# Build a model object with the default environment
with gp.Model(name="model") as model:
    pass

model = gp.Model(name="model")
model.dispose()
gp.disposeDefaultEnv()

# Build a model object with a new environment
with gp.Env() as env, gp.Model(name="model", env=env) as model:
    pass

env = gp.Env()
model = gp.Model(name="model", env=env)
model.dispose()
env.dispose()

## Decision Variables

Since variables are associated with a particular model object, use the [Model.addVar()](https://www.gurobi.com/documentation/current/refman/py_model_addvar.html#pythonmethod:Model.addVar) method to create a Gurobi variable object ([Var](https://www.gurobi.com/documentation/current/refman/py_var.html)):
```
Model.addVar(lb=0, ub=float("inf"), obj=0, vtype=GRB.CONTINUOUS, name="", column=None)
```

The available variable types in Gurobi are:
- Continuous: `GRB.CONTINUOUS`
- General integer: `GRB.INTEGER`
- Binary: `GRB.BINARY`
- Semi-continuous: `GRB.SEMICONT`
- Semi-integer: `GRB.SEMIINT`

A semi-continuous variable has the property that it takes a value of 0, or a value between the specified lower and upper bounds. A semi-integer variable adds the additional restriction that the variable should take an integral value.

In [ ]:
import gurobipy as gp
from gurobipy import GRB

with gp.Model(name="model") as model:
    # Define a binary decision variable
    x = model.addVar(vtype=GRB.BINARY, name="x")
    # Define an integer variable with lb=-1, ub=100
    y = model.addVar(lb=-1, ub=100, vtype=GRB.INTEGER, name="y")

## [Model.addVars()](https://www.gurobi.com/documentation/current/refman/py_model_addvars.html#pythonmethod:Model.addVar)

To add multiple decision variables to the model, use the Model.addVars() method which returns a Gurobi tupledict object containing the newly created variables:
```
Model.addVars(*indices, lb=0.0, ub=float('inf'), obj=0.0, vtype=GRB.CONTINUOUS, name="")
```
The first argument is indices for accessing the variables:
- Integers
- lists of scalars
- tuplelist
- generator

The given name is subscripted by the index of the generator expression. 
- The names are stored as ASCII strings 
    - avoid using names that contain non-ASCII characters and spaces

In [ ]:
import gurobipy as gp
from gurobipy import GRB

with gp.Model(name="model") as model:
    # 3D array of binary variables
    x = model.addVars(2, 3, 4, vtype=GRB.BINARY, name="x")
    model.update()
    print(model.getAttr("VarName", model.getVars()))

    # Use arbitrary lists of immutable objects to create a tupledict of 6 variables
    y = model.addVars([1, 5], [7, 3, 2], ub=range(6), name=[f"y_{i}" for i in range(6)])
    model.update()
    print("\nVariables names, upper bounds, and indices:")
    for index, var in y.items():
        print(f"name: {var.VarName}, ub: {var.UB}, index: {index}")

    # Use arbitrary list of tuples as indices
    z = model.addVars(
        [(3, "a"), (3, "b"), (7, "b"), (7, "c")],
        lb=-GRB.INFINITY,
        ub=GRB.INFINITY,
        name="z",
    )
    model.update()
    print("\nVariables names and lower and upper bounds:")
    for index, var in z.items():
        print(f"name: {var.VarName}, lb: {var.LB}, ub: {var.UB}")

## Constraints
Like variables, constraints are also associated with a model. Use the method [Model.addConstr()](https://www.gurobi.com/documentation/current/refman/py_model_addconstr.html) to add a constraint to a model.
```
Model.addConstr(constr, name="")
```

`constr` is a [TempConstr](https://www.gurobi.com/documentation/current/refman/py_tempconstr.html#pythonclass:TempConstr) object that can take different types:

- Linear Constraint: `x + y <= 1` 
- Ranged Linear Constraint: `x + y == [1, 3]`
- Quadratic Constraint: `x*x + y*y + x*y <= 1`
- Linear Matrix Constraint: `A @ x <= 1`
- Quadratic Matrix Constraint: `x @ Q @ y <= 2`
- Absolute Value Constraint: `x == abs_(y)`
- Logical Constraint: `x == and_(y, z)`
- Min or Max Constraint: `z == max_(x, y, constant=9)`
- Indicator Constraint: `(x == 1) >> (y + z <= 5)`

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Add constraint "\sum_{i=1}^{n} x_i <= b" for any given n and b. Assume x_i are binary variables
n, b = 10, 4
with gp.Model("model") as model:
    x = model.addVars(n, vtype=GRB.BINARY, name="x")
    model.addConstr(gp.quicksum(x[i] for i in range(n)) <= b, name="c1")
    model.update()

    # Print the LHS, Sense, and RHS of c1
    c1 = model.getConstrByName("c1")
    print(f"RHS, sense = {c1.RHS}, {c1.Sense}")
    print(f"row: {model.getRow(c1)}")
    print("\n\n")

# Add constraints "x_i + y_j - x_i*y_j >= 3". Asssume x_i and y_j are continuous
n, m = 5, 4
with gp.Model("model") as model:
    x = model.addVars(n, name="x")
    y = model.addVars(m, name="y")

    for i in range(n):
        for j in range(m):
            model.addConstr(x[i] + y[j] - x[i] * y[j] >= 3, name=f"c_{i}{j}")

    model.update()

    # Print the LHS, Sense, and RHS of all c_ij constraints
    for c in model.getQConstrs():
        print(f"Name: {c.QCName}")
        print(f"\tRHS, sense = {c.QCRHS}, {c.QCSense}")
        print(f"\trow: {model.getQCRow(c)}")

## [Model.addConstrs](https://www.gurobi.com/documentation/current/refman/py_model_addconstrs.html)

To add multiple constraints to the model, use the Model.addConstrs() method which returns a Gurobi tupledict that contains the newly created constraints:

```
Model.addConstrs(generator, name="")
```

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Add constraints x_i + y_j <= 1 for all (i, j). Assume x_i and y_j are binary variables
I = range(5)
J = ["a", "b", "c"]
with gp.Model("model") as model:
    x = model.addVars(I, vtype=GRB.BINARY, name="x")
    y = model.addVars(J, vtype=GRB.BINARY, name="y")

    generator = (x[i] + y[j] <= 1 for i in I for j in J)
    model.addConstrs(generator, name="c")
    model.update()

    # Print constraint names
    print(model.getAttr("ConstrName", model.getConstrs()))

## Objective Function

To set the model objective equal to a linear or a quadratic expression, use the [Model.setObjective()](https://www.gurobi.com/documentation/current/refman/py_model_setobjective.html) method:
```
Model.setObjective(expr, sense=GRB.MINIMIZE)
```
- expr: 
    - [LinExpr()](https://www.gurobi.com/documentation/current/refman/py_lex.html): constant + coefficient-variable pairs capturing linear terms
    - [QuadExpr()](https://www.gurobi.com/documentation/current/refman/py_qex.html): linear expression + list of coefficient-variable-variable triples
- sense:
    - GRB.MINIMIZE (default) or GRB.MAXIMIZE

In [ ]:
import gurobipy as gp
from gurobipy import GRB

import numpy as np

# Add linear objectives in the form c^Tx and quadratic objective in the form x^T Q x
n = 10
c = np.random.rand(n)
Q = np.random.rand(n, n)

with gp.Model("model") as model:
    x = model.addVars(n, name="x")
    linexpr = gp.quicksum(c[i] * x[i] for i in range(n))
    # linexpr = gp.quicksum(c_i * x_i for c_i, x_i in zip(c, x.values()))
    model.setObjective(linexpr)
    model.update()

    # Print objective expression
    obj = model.getObjective()
    print(f"obj: {obj}")

with gp.Model("model") as model:
    x = model.addVars(n, name="x")
    quadexpr = 0
    for i in range(n):
        for j in range(n):
            quadexpr += x[i] * Q[i, j] * x[j]
    model.setObjective(quadexpr)
    model.update()

    # Print objective expression
    obj = model.getObjective()
    print(f"\nobj: {obj}")

## [Attributes](https://www.gurobi.com/documentation/current/refman/attributes.html)

The primary mechanism for querying and modifying properties of a Gurobi object is through the attribute interface. You can see the complete set of Gurobi attributes in the reference linked above.

Let us see an example of how to query useful attributes on the model object after the optimization is complete.

In [ ]:
import urllib.request
import gurobipy as gp
from gurobipy import GRB

url = "https://raw.githubusercontent.com/Gurobi/modeling-examples/master/intro_to_gurobipy/data/glass4.mps.bz2"
path_to_file, _ = urllib.request.urlretrieve(url, "./glass4.mps.bz2")

# running notebook locally:
# with gp.read("data/glass4.mps.bz2") as model:
with gp.read(path_to_file) as model:
    model.optimize()

    print("****************** SOLUTION ******************")
    print(f"\tStatus       : {model.Status}")
    print(f"\tObj          : {model.ObjVal}")
    print(f"\tSolutionCount: {model.SolCount}")
    print(f"\tRuntime      : {model.Runtime}")
    print(f"\tMIPGap       : {model.MIPGap}")

    print("\n")
    for var in model.getVars()[:20]:
        print(f"\t{var.VarName} = {var.X}")

## [Parameters](https://www.gurobi.com/documentation/current/refman/parameter_descriptions.html)
Parameters control the mechanics of the Gurobi Optimizer.

In [ ]:
import urllib.request
import gurobipy as gp
from gurobipy import GRB

url = "https://raw.githubusercontent.com/Gurobi/modeling-examples/master/intro_to_gurobipy/data/glass4.mps.bz2"
path_to_file, _ = urllib.request.urlretrieve(url, "./glass4.mps.bz2")

# running notebook locally:
# with gp.read("data/glass4.mps.bz2") as model:
with gp.read(path_to_file) as model:
    model.params.Threads = 1
    model.params.TimeLimit = 10
    model.optimize()

## [General Constraints](https://www.gurobi.com/documentation/current/refman/constraints.html#subsubsection:GeneralConstraints)

General constraints allow defining certain variable relationships easily without going to the hardship of modeling these relationships in terms of the more fundamental constraints of MIP. Capturing a single one of these general constraints can often require a large set of constraints and a number of auxiliary decision variables.

- __Simple General Constraints__: 
    - $z = \mbox{max(x, y, 3)}$: `model.addConstr(z == max_(x, y, constant=3))`
    - $z = \mbox{min(x, y, 3)}$: `model.addConstr(z == min_(x, y, constant=3))`
    - $y = \mbox{abs(x)}$: `model.addConstr(y == abs_(x))`
    - $z = x \land y$: `model.addConstr(z == and_(x, y))`
    - $z = x \lor y$: `model.addConstr(z == or_(x, y))`
    - $z = ||x||_p, ~~ p = 0, 1, 2, \infty$: `model.addConstr(nx == norm(x, 1.0))`
    - indicator: 
        -  $x_0 = 1 -> x_1 + 2 x_2 + x_3 \leq 1$
            - `model.addGenConstrIndicator(x0, 1, x1 + 2*x2 + x3 <= 1)`
            - `model.addConstr((x0 == 1) >> (x1 + 2*x2 + x3 <= 1))`
    - piece-wise linear:
        - `model.addGenConstrPWL(x, y, [0, 1, 2], [1.5, 0, 3], "")`
- __Function Constraints__: 
    - $y = p_0x^n + p_1x^{n-1} + \ldots + p_nx+ p_{n+1}$:
        - $y = 2 x^3 + 1.5 x^2 + 1$
          - `model.addGenConstrPoly(x, y, [2, 1.5, 0, 1])`
    - $y = e^x$: `model.addGenConstrExp(x, y)`
    - $y = a^x$: `model.addGenConstrExpA(x, y, a)`
    - $y = \ln(x)$: `model.addGenConstrLog(x, y)`
    - $y = \log_a(x)$: `model.addGenConstrLogA(x, y, a)`
    - $y = \frac{1}{1+e^{-x}}$: `model.addGenConstrLogistic(x, y)`
    - $y = x^a$: `model.addGenConstrPow(x, y, a)`
    - $y = \sin(x)$: `model.addGenConstrSin(x, y)`
    - $y = \cos(x)$: `model.addGenConstrCos(x, y)`
    - $y = \tan(x)$: `model.addGenConstrTan(x, y)`
   

Gurobi will automatically add a piecewise-linear approximation of the function to the model. 

In [ ]:
# Consider the following nonconvex nonlinear problem
#
#  maximize    2 x    + y
#  subject to  exp(x) + 4 sqrt(y) <= 9
#              x, y >= 0

import gurobipy as gp
from gurobipy import GRB
import math

with gp.Model("model") as model:
    x = model.addVar(name="x")
    y = model.addVar(name="y")
    u = model.addVar(name="u")
    v = model.addVar(name="v")

    # Set objective
    model.setObjective(2 * x + y, GRB.MAXIMIZE)

    # u = exp(x)
    gcf1 = model.addGenConstrExp(x, u, name="gcf1")
    # v = y^(0.5)
    gcf2 = model.addGenConstrPow(y, v, 0.5, name="gcf2")
    c = model.addConstr(u + 4 * v <= 9)

    # Use the equal piece length approach with the length = 1e-3
    model.Params.FuncPieces = 1
    model.Params.FuncPieceLength = 1e-3

    # Optimize the model
    model.optimize()

    print("****************** SOLUTION ******************")
    print(f"x = {x.X}, u = {u.X}")
    print(f"y = {y.X}, v = {v.X}")
    print(f"Obj = {model.ObjVal}")

    # Calculate violation of exp(x) + 4 sqrt(y) <= 9
    vio = math.exp(x.X) + 4 * math.sqrt(y.X) - 9
    if vio < 0:
        vio = 0
    print(f"Vio = {vio}")

## Example: Portfolio Optimization

In a portfolio optimization problem, there are $n$ assets. Each asset $i$ is associated with an expected return $\mu_i$ and each pair of assets $(i, j)$ has a covariance (risk) $\sigma_{ij}$. The goal is to find the optimal fraction of the portfolio invested in each asset to minimize the risk of investment such that 1) the total expected return of the investment exceeds the minimum target return $\mu_0$ and 2) the portfolio invests in at most $k \leq n$ assets.

- $x_i$: Relative investment in asset $i$
- $y_i$: Binary variable controlling whether asset $i$ is traded

\begin{align}
\mbox{minimize} \quad & \sum_{i=1}^{n} \sum_{j=1}^{n} \sigma_{ij} x_i x_j & \notag \\
\mbox{subject to} \quad & \sum_{i=1}^{n} \mu_i x_i \geq \mu_0 & \notag \\
                        & \sum_{i=1}^{n} x_i = 1 & \notag \\
                        & \sum_{i=1}^{n} y_i \leq k & \notag \\
                        & x_i \leq y_i & i=1, \ldots, n \notag \\
                        & 0 \leq x_i \leq 1 & i=1, \ldots, n \notag \\
                        & y_i \in \{0, 1\} & i=1, \ldots, n \notag
\end{align}

In [ ]:
import json
from urllib.request import urlopen
import numpy as np

url = "https://raw.githubusercontent.com/Gurobi/modeling-examples/master/intro_to_gurobipy/data/portfolio-example.json"
response = urlopen(url)
data = json.loads(response.read())

n = data["num_assets"]
sigma = np.array(data["covariance"])
mu = np.array(data["expected_return"])
mu_0 = data["target_return"]
k = data["portfolio_max_size"]

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Write the solution to a pandas.DataFrame
import pandas as pd

with gp.Model("term-based") as model:
    x = model.addVars(n, ub=1, name="x")
    y = model.addVars(n, vtype=GRB.BINARY, name="y")

    risk = gp.quicksum(x[i] * sigma[i, j] * x[j] for i in range(n) for j in range(n))
    # Another approach to build the risk expression
    # risk = gp.QuadExpr()
    # for i in range(n):
    #    for j in range(n):
    #        risk.addTerms(sigma[i, j], x[i], x[j])
    model.setObjective(risk)

    expected_return = gp.quicksum(mu[i] * x[i] for i in range(n))
    model.addConstr(expected_return >= mu_0, name="return")

    model.addConstr(x.sum() == 1, name="budget")
    # Another approach to build the budget constraint
    # model.addConstr(gp.quicksum(x[i] for i in range(n)) == 1, name="budget")

    model.addConstr(y.sum() <= k, name="cardinality")

    model.addConstrs((x[i] <= y[i] for i in range(n)), name="is_allocated")

    model.optimize()

    # Write the solution into a DataFrame
    portfolio = [var.X for var in model.getVars() if "x" in var.VarName]
    risk = model.ObjVal
    expected_return = model.getRow(model.getConstrByName("return")).getValue()
    df = pd.DataFrame(
        data=portfolio + [risk, expected_return],
        index=[f"asset_{i}" for i in range(n)] + ["risk", "return"],
        columns=["Portfolio"],
    )

    print(df)

## Best Practices

- Do ensure the separation between your model and data
- Do use descriptive names where appropriate
- Exploit sparsity when creating variables/constraints - Only create variables/constraints for valid combinations
- Don’t forget to dispose of your model and environment
- Use the documentation

## [Card Game](https://www.gurobi.com/resources/optimization-gamification-introducing-the-gurobipy-card-game/)


<img src="./images/card-game.png" width="800" height="600" style="margin-left:auto; margin-right:auto"/>


## Resources
- [Gurobi Python Documentation](https://www.gurobi.com/documentation/current/refman/py_python_api_overview.html)
- [Gurobi Python Examples](https://www.gurobi.com/documentation/current/examples/python_examples.html)
- [Gurobi Jupyter Notebook Modeling Examples](https://www.gurobi.com/jupyter_models/)
- [Gurobi Knowledge Base](https://support.gurobi.com/hc/en-us/categories/360000840331-Knowledge-Base)